In [63]:
import pandas as pd
import numpy as np
from IPython.display import display

import warnings
warnings.filterwarnings('ignore')

data = pd.read_csv("../datasets/event_users_train.csv")

print "There are %d rows in this dataframe" %len(data)

data.head()

There are 2268290 rows in this dataframe


,user_id,monster_id,count_monster
0,5287577520,2080,3
1,5289159737,1039,29
2,5318181007,2008,3
3,5414546335,2081,3
4,5413724326,2005,8


In [64]:
data.describe()

,user_id,monster_id,count_monster
count,2.268290e+06,2.268290e+06,2.268290e+06
mean,5.499524e+09,1.832621e+03,3.822632e+00
std,2.890100e+08,4.383383e+02,9.572808e+00
min,5.000018e+09,1.001000e+03,1.000000e+00
25%,5.248962e+09,2.002000e+03,1.000000e+00
50%,5.497722e+09,2.023000e+03,2.000000e+00
75%,5.751172e+09,2.083000e+03,4.000000e+00
max,5.999992e+09,2.284000e+03,2.327000e+03


In [65]:
data.loc[data['count_monster'] > 30, 'count_monster'] = 30
 
data.describe()

,user_id,monster_id,count_monster
count,2.268290e+06,2.268290e+06,2.268290e+06
mean,5.499524e+09,1.832621e+03,3.503286e+00
std,2.890100e+08,4.383383e+02,4.535809e+00
min,5.000018e+09,1.001000e+03,1.000000e+00
25%,5.248962e+09,2.002000e+03,1.000000e+00
50%,5.497722e+09,2.023000e+03,2.000000e+00
75%,5.751172e+09,2.083000e+03,4.000000e+00
max,5.999992e+09,2.284000e+03,3.000000e+01


In [66]:
from sklearn import preprocessing

import warnings
warnings.filterwarnings('ignore')

scaler = preprocessing.MinMaxScaler()
count_scaled = scaler.fit_transform(data['count_monster'])
data['rating'] = pd.DataFrame(count_scaled)

data.head()

,user_id,monster_id,count_monster,rating
0,5287577520,2080,3,0.068966
1,5289159737,1039,29,0.965517
2,5318181007,2008,3,0.068966
3,5414546335,2081,3,0.068966
4,5413724326,2005,8,0.241379


<h1>Monsters info</h1>

In [67]:
import pandas as pd
import numpy as np
from IPython.display import display

monsters_info = pd.read_csv("../datasets/monsters_info2.csv")

print "There are %d rows in this dataframe" %len(monsters_info)

monsters_info.head()

There are 148 rows in this dataframe


,monster_id,monster_name,attributes,rarity,breedable,price,lvl
0,1015,Sealion,"[f, w]",2,Yes,360,3
1,1017,Thundenix,"[t, f]",3,Yes,280,3
2,1018,Scorchpeg,"[f, l]",4,Yes,1700,3
3,1019,Firetaur,"[d, f]",3,Yes,560,3
4,1020,Djinn,"[m, f]",3,Yes,120,3


In [68]:
monsters_info = pd.read_csv("../datasets/monsters_info2.csv")
monsters_ids = monsters_info['monster_id']

print "There are %d rows in monsters_ids" %len(monsters_ids)

monsters_ids.head()

There are 148 rows in monsters_ids


0    1015
1    1017
2    1018
3    1019
4    1020
Name: monster_id, dtype: int64

In [69]:
# Let's create a function that collect the monster preferences of our common players
def get_monsters_reviews(monster_id, common_players):
    mask = (data.user_id.isin(common_players)) & (data.monster_id==monster_id)
    reviews = data[mask].sort('user_id')
    reviews = reviews[reviews.user_id.duplicated()==False]
    return reviews

In [70]:
# Importing the scipy library to measure the pearson correlation coefficient
from scipy.stats.stats import pearsonr
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings('ignore')

def calculate_correlation(monster1, monster2):
    # We start by finding the common players
    monster_1_players = data[data.monster_id == monster1].user_id
    monster_2_players = data[data.monster_id == monster2].user_id

    common_players = set(monster_1_players).intersection(monster_2_players)

    # Then we look for the reviews given by common reviewers
    monster_1_reviews = get_monsters_reviews(monster1, common_players)
    monster_2_reviews = get_monsters_reviews(monster2, common_players)
#    if monster_1_reviews.size == 0:
#        return 0.
#    print monster_1_reviews.size
    # Calculate the Pearson Correlation Score
    return pearsonr(monster_1_reviews.rating, monster_2_reviews.rating)[0]
    # return manhattan_distances(monster_1_reviews.rating, monster_2_reviews.rating)[0][0]
    # return cosine_similarity(monster_1_reviews.rating, monster_2_reviews.rating)

In [71]:
# Top 20 most used monsters 
top_monsters = pd.value_counts(data.monster_id)
top_20_monsters = top_monsters[:20]
top_monsters = top_monsters[:20]
# for top_monster in top_20_monsters.iteritems():
#    print top_monster[0]

In [72]:
from time import time

import warnings
warnings.filterwarnings('ignore')

# calculate the correlation for our top books
correlation_coefficient = []

for top_monster_1 in top_monsters.iteritems():
    monster_1_id = top_monster_1[0]
    start = time()
    print "Start correlation monster: ", monster_1_id
    for top_monster_2 in top_monsters.iteritems():
        monster_2_id = top_monster_2[0]
        if monster_1_id != monster_2_id:
            row = [monster_1_id, monster_2_id] + [calculate_correlation(monster_1_id, monster_2_id)]
            correlation_coefficient.append(row)
    end = time()
    print "\t... done in {:.4f} seconds.".format(end - start)

Start correlation monster:  1031
	... done in 15.4842 seconds.
Start correlation monster:  1039
	... done in 15.7056 seconds.
Start correlation monster:  1045
	... done in 15.4744 seconds.
Start correlation monster:  2017
	... done in 15.4904 seconds.
Start correlation monster:  2019
	... done in 15.3287 seconds.
Start correlation monster:  1017
	... done in 15.3724 seconds.
Start correlation monster:  2014
	... done in 15.3436 seconds.
Start correlation monster:  1032
	... done in 15.3532 seconds.
Start correlation monster:  2078
	... done in 15.1840 seconds.
Start correlation monster:  2079
	... done in 15.1898 seconds.
Start correlation monster:  1051
	... done in 15.3027 seconds.
Start correlation monster:  2077
	... done in 15.5162 seconds.
Start correlation monster:  1044
	... done in 16.8974 seconds.
Start correlation monster:  1052
	... done in 24.8119 seconds.
Start correlation monster:  2021
	... done in 31.1712 seconds.
Start correlation monster:  2018
	... done in 29.0344 s

In [73]:
import pickle
pickle.dump(correlation_coefficient, open( "correlation_coefficient_13.p", "wb" ) )

In [74]:
# Let's look at what the table of correlation looks like
cols = ["Monster_1", "Monster_2", "Correlation"]
correlation_coefficient = pd.DataFrame(correlation_coefficient, columns=cols).sort('Correlation', ascending=False)
correlation_coefficient.head()

,Monster_1,Monster_2,Correlation
11,1031,1044,0.875952
228,1044,1031,0.875952
31,1039,1052,0.779704
248,1052,1039,0.779704
142,1032,1051,0.758424


In [75]:
correlation_coefficient.describe()

,Monster_1,Monster_2,Correlation
count,380.000000,380.000000,380.000000
mean,1643.250000,1643.250000,0.157394
std,494.769836,494.769836,0.141350
min,1017.000000,1017.000000,-0.050421
25%,1044.750000,1044.750000,0.063437
50%,2015.500000,2015.500000,0.134254
75%,2072.250000,2072.250000,0.240862
max,2082.000000,2082.000000,0.875952
